
# FSP Deterministic Hydrology Regression

This notebook reproduces the single–well deterministic hydrology scenario from the legacy MATLAB Fault Slip Potential (FSP) tool and compares the Julia web tool implementation against the MATLAB results stored under `regression_tests/gold`. The workflow mirrors the logic in `FSP_3/deterministic_hydrology_process.jl` with every hydrology modeling functionality invoking the existing Julia modules that are used in the web tool (3.0 version).



## Prerequisites

1. (Optional) Activate the dedicated environment and install notebook dependencies:
   ```bash
   conda activate julia_jupyter_venv
   pip install nbformat notebook pandas numpy matplotlib
   ```
2. Ensure the `julia` executable is on your `PATH` and that `Pkg.instantiate()` has been run for the `FSP_3` project. `Pkg.instantiate()` ensures that the active project's dependencies are correctly installed and available. 
3. From the repository root, launch Jupyter Lab / Notebook and open this notebook.

> The notebook does not modify any Julia files; it only references them via `include(...)` calls inside temporary Julia snippets.


In [ ]:

%matplotlib inline
import json
import math
import os
import subprocess
import textwrap
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

plt.style.use('seaborn-v0_8-darkgrid')


In [ ]:

REPO_ROOT = Path.cwd().resolve()
FSP_ROOT = REPO_ROOT / "FSP_3"
GOLD_DIR = REPO_ROOT / "regression_tests" / "gold"
TMP_DIR = REPO_ROOT / "regression_tests" / ".fsp_regression_tmp"
TMP_DIR.mkdir(parents=True, exist_ok=True)

KM_PER_DEG = 111.32  # approximate conversion at the equator

YEARS_TO_TEST = [2018, 2019, 2020, 2021]

SCENARIO = {
    "aquifer": {
        "thickness_ft": 100.0,
        "porosity": 0.10,
        "permeability_md": 200.0,
    },
    "fluid": {
        "density": 1000.0,
        "viscosity": 0.0008,
        "compressibility": 3.6e-10,
        "gravity": 9.81,
    },
    "rock": {
        "compressibility": 1.08e-9,
    },
    "grid": {
        "x_min_km": 2.0,
        "x_max_km": 18.0,
        "y_min_km": 2.0,
        "y_max_km": 18.0,
        "npts": 50,
    },
    "injection": {
        "start_year": 2017,
        "end_year": 2020,
        "rate_bpd": 25_000.0,
        "well_id": "WELL-001",
        "data_type": "monthly_fsp",
        "extrapolate": False,
        "well_x_km": 10.0,
        "well_y_km": 10.0,
    },
}
SCENARIO["injection"]["well_lat"] = SCENARIO["injection"]["well_y_km"] / KM_PER_DEG
SCENARIO["injection"]["well_lon"] = SCENARIO["injection"]["well_x_km"] / KM_PER_DEG



## Load MATLAB reference surfaces

The gold CSV files correspond to annual pressure grids exported from FSP v2.0. Each surface spans a 50×50 grid from 2–18 km in both easting and northing.


In [ ]:

def load_gold_surfaces(years):
    frames = []
    for year in years:
        csv_path = GOLD_DIR / f"ConstantInjectionReference{year}.csv"
        df = pd.read_csv(csv_path, skiprows=1)
        df = df.rename(columns={
            "X_Easting_km": "X_Easting_km",
            " Y_Northing_km": "Y_Northing_km",
            " additionalPressure_PSI": "additionalPressure_PSI",
        })
        df["Year"] = year
        frames.append(df)
    gold = pd.concat(frames, ignore_index=True)
    gold = gold.sort_values(["Year", "Y_Northing_km", "X_Easting_km"]).reset_index(drop=True)
    return gold

GOLD_DF = load_gold_surfaces(YEARS_TO_TEST)
GOLD_DF.head()



## Build the monthly injection schedule

We replicate the reference case: one well injecting 25 kbpd from Jan 2017 through Dec 2018, then shut-in afterwards.


In [ ]:

def build_monthly_schedule(scenario):
    records = []
    start = pd.Timestamp(f"{scenario['injection']['start_year']}-01-01")
    end = pd.Timestamp(f"{scenario['injection']['end_year']}-12-01")
    current = start
    while current <= end:
        if current < pd.Timestamp("2019-01-01"):
            rate_bpd = scenario["injection"]["rate_bpd"]
        else:
            rate_bpd = 0.0
        records.append({
            "WellID": scenario["injection"]["well_id"],
            "Latitude(WGS84)": scenario["injection"]["well_lat"],
            "Longitude(WGS84)": scenario["injection"]["well_lon"],
            "Year": current.year,
            "Month": current.month,
            "InjectionRate(bbl/month)": rate_bpd * current.days_in_month,
        })
        current += pd.offsets.MonthBegin(1)
    return pd.DataFrame(records)

INJECTION_DF = build_monthly_schedule(SCENARIO)
INJECTION_DF.head()



## Helper: run Julia deterministic hydrology

The function below writes the injection schedule and scenario configuration to temporary files, then runs a short Julia snippet. That snippet includes:

- `FSP_3/core/utilities.jl`
- `FSP_3/core/hydrology_calculations.jl`

It mirrors the computations inside `deterministic_hydrology_process.jl`: computing storativity/transmissivity, building the spatial grid, preparing well histories via `prepare_well_data_for_pressure_scenario`, and evaluating `pfieldcalc_all_rates`. The output is a flattened CSV with `(Year, Latitude, Longitude, Pressure_psi)` entries.


In [ ]:

def run_fsp_det_hydrology(years, scenario, injection_df, tmp_dir):
    inj_csv = tmp_dir / "injection_monthly.csv"
    injection_df.to_csv(inj_csv, index=False)

    config = {
        "aquifer": scenario["aquifer"],
        "fluid": scenario["fluid"],
        "rock": scenario["rock"],
        "grid": scenario["grid"],
        "injection": {
            **scenario["injection"],
            "well_id": scenario["injection"]["well_id"],
            "well_lat": scenario["injection"]["well_lat"],
            "well_lon": scenario["injection"]["well_lon"],
        },
        "years_to_run": years,
    }

    config_path = tmp_dir / "fsp_regression_config.json"
    with open(config_path, "w", encoding="utf-8") as fp:
        json.dump(config, fp, indent=2)

    output_csv = tmp_dir / "fsp_pressure_output.csv"

    julia_script = textwrap.dedent(""
        using CSV, DataFrames, Dates, JSON3
        include("FSP_3/core/utilities.jl")
        include("FSP_3/core/hydrology_calculations.jl")

        const KM_PER_DEG = 111.32

        km_to_deg(km) = km / KM_PER_DEG

        function build_grid(cfg)
            lat_min = km_to_deg(cfg["grid"]["y_min_km"])
            lat_max = km_to_deg(cfg["grid"]["y_max_km"])
            lon_min = km_to_deg(cfg["grid"]["x_min_km"])
            lon_max = km_to_deg(cfg["grid"]["x_max_km"])
            Utilities.create_spatial_grid_latlon(lat_min, lat_max, lon_min, lon_max, cfg["grid"]["npts"])
        end

        function run_regression(config_path::String, injection_csv::String, output_csv::String)
            cfg = JSON3.read(read(config_path, String))
            inj_df = CSV.read(injection_csv, DataFrame)
            LAT_grid, LON_grid, _, _ = build_grid(cfg)
            S, T, _ = HydroCalculations.calcST(
                cfg["aquifer"]["thickness_ft"],
                cfg["aquifer"]["porosity"],
                cfg["aquifer"]["permeability_md"],
                cfg["fluid"]["density"],
                cfg["fluid"]["viscosity"],
                cfg["fluid"]["gravity"],
                cfg["fluid"]["compressibility"],
                cfg["rock"]["compressibility"],
            )
            STRho = (S, T, cfg["fluid"]["density"])

            results = DataFrame(Year=Int[], Latitude=Float64[], Longitude=Float64[], Pressure_psi=Float64[])
            well_id = cfg["injection"]["well_id"]
            inj_start_year = cfg["injection"]["start_year"]
            inj_end_year = cfg["injection"]["end_year"]
            inj_start_date = Date(inj_start_year, 1, 1)
            inj_end_date = Date(inj_end_year, 12, 31)

            for year in cfg["years_to_run"]
                if year < inj_start_year
                    continue
                end
                yoi_date = Date(year - 1, 12, 31)
                year_end = min(inj_end_year, year)
                days, rates = Utilities.prepare_well_data_for_pressure_scenario(
                    inj_df,
                    string(well_id),
                    inj_start_year,
                    inj_start_date,
                    year_end,
                    inj_end_date,
                    cfg["injection"]["data_type"],
                    year,
                    cfg["injection"]["extrapolate"],
                    yoi_date,
                )
                if length(days) == 0
                    continue
                end

                eval_days = Float64((yoi_date - inj_start_date).value + 1)
                pfield = HydroCalculations.pfieldcalc_all_rates(
                    LAT_grid,
                    LON_grid,
                    STRho,
                    days,
                    rates,
                    cfg["injection"]["well_lon"],
                    cfg["injection"]["well_lat"],
                    "latlon",
                    eval_days,
                )

                latitudes = vec(LON_grid)
                longitudes = vec(LAT_grid)
                pressures = vec(pfield)

                year_df = DataFrame(
                    Year = fill(year, length(pressures)),
                    Latitude = latitudes,
                    Longitude = longitudes,
                    Pressure_psi = pressures,
                )
                append!(results, year_df)
            end

            CSV.write(output_csv, results)
        end

        run_regression("<<CONFIG_PATH>>", "<<INJECTION_CSV>>", "<<OUTPUT_CSV>>")
    "")

    julia_script = julia_script.replace("<<CONFIG_PATH>>", str(config_path.as_posix()))
    julia_script = julia_script.replace("<<INJECTION_CSV>>", str(inj_csv.as_posix()))
    julia_script = julia_script.replace("<<OUTPUT_CSV>>", str(output_csv.as_posix()))

    cmd = [
        "julia",
        f"--project={FSP_ROOT.as_posix()}",
        "-e",
        julia_script,
    ]

    print("Running Julia deterministic hydrology...", flush=True)
    try:
        completed = subprocess.run(
            cmd,
            cwd=REPO_ROOT,
            check=True,
            capture_output=True,
            text=True,
        )
    except FileNotFoundError as exc:
        raise RuntimeError("Could not find the `julia` executable on PATH. Activate the julia_jupyter_venv environment and ensure Julia is installed.") from exc

    if completed.stdout:
        print(completed.stdout)
    if completed.stderr:
        print(completed.stderr)

    return output_csv



## Execute the Julia regression run

> This cell launches Julia. If Julia is not installed or the environment is missing packages, fix that before proceeding.


In [ ]:

OUTPUT_CSV = run_fsp_det_hydrology(YEARS_TO_TEST, SCENARIO, INJECTION_DF, TMP_DIR)
FSP_DF = pd.read_csv(OUTPUT_CSV)
FSP_DF.head()



## Compare MATLAB vs Julia pressure grids

We align both datasets on the same 50×50 grid (converted back to kilometers) and compute basic regression metrics per year.


In [ ]:

def build_pressure_grids(df, value_col, lat_col, lon_col):
    grids = {}
    df = df.copy()
    df = df.sort_values(["Year", lat_col, lon_col])
    for year, group in df.groupby("Year"):
        pivot = group.pivot(index=lat_col, columns=lon_col, values=value_col)
        grids[year] = pivot.sort_index().sort_index(axis=1)
    return grids

GOLD_DF["X_km"] = GOLD_DF["X_Easting_km"]
GOLD_DF["Y_km"] = GOLD_DF["Y_Northing_km"]
GOLD_GRIDS = build_pressure_grids(GOLD_DF, "additionalPressure_PSI", "Y_km", "X_km")

FSP_DF["X_km"] = FSP_DF["Longitude"] * KM_PER_DEG
FSP_DF["Y_km"] = FSP_DF["Latitude"] * KM_PER_DEG
FSP_GRIDS = build_pressure_grids(FSP_DF, "Pressure_psi", "Y_km", "X_km")

metrics = []
for year in YEARS_TO_TEST:
    if year not in GOLD_GRIDS or year not in FSP_GRIDS:
        continue
    gold_grid = GOLD_GRIDS[year]
    fsp_grid = FSP_GRIDS[year].reindex(index=gold_grid.index, columns=gold_grid.columns)
    diff = fsp_grid.to_numpy() - gold_grid.to_numpy()
    metrics.append({
        "Year": year,
        "RMSE (psi)": float(np.sqrt(np.mean(diff ** 2))),
        "Max |Δ| (psi)": float(np.max(np.abs(diff))),
        "Median |Δ| (psi)": float(np.median(np.abs(diff))),
    })

METRICS_DF = pd.DataFrame(metrics)
METRICS_DF



## Visual comparison

The plots below show (1) MATLAB gold, (2) Julia FSP, and (3) the difference surface for each target year.


In [ ]:

fig, axes = plt.subplots(len(YEARS_TO_TEST), 3, figsize=(15, 4 * len(YEARS_TO_TEST)))
for row, year in enumerate(YEARS_TO_TEST):
    if year not in GOLD_GRIDS or year not in FSP_GRIDS:
        for col in range(3):
            axes[row, col].axis('off')
        axes[row, 0].set_title(f"Year {year} missing")
        continue
    gold_grid = GOLD_GRIDS[year]
    fsp_grid = FSP_GRIDS[year].reindex(index=gold_grid.index, columns=gold_grid.columns)
    diff_grid = fsp_grid - gold_grid

    for col, (title, data, cmap) in enumerate([
        ("MATLAB", gold_grid, 'viridis'),
        ("Julia", fsp_grid, 'viridis'),
        ("Julia - MATLAB", diff_grid, 'RdBu_r'),
    ]):
        ax = axes[row, col]
        im = ax.imshow(
            data,
            origin='lower',
            extent=[gold_grid.columns.min(), gold_grid.columns.max(), gold_grid.index.min(), gold_grid.index.max()],
            aspect='equal',
            cmap=cmap,
        )
        fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
        ax.set_title(f"{title} {year}")
        ax.set_xlabel('X (km)')
        ax.set_ylabel('Y (km)')

plt.tight_layout()
plt.show()



## Next steps

- Add additional scenarios (multiple wells, different rate schedules) to widen coverage.
- Integrate pass/fail thresholds (e.g., RMSE < 0.5 psi) so the regression can be run in CI.
- Extend the notebook to validate hydrology-driven fault pressures once portal-style fault datasets are available.
